In [84]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
from sklearn.preprocessing import OneHotEncoder

def fetch_data(url):
    response = requests.get(url)
    data = StringIO(response.text)
    df = pd.read_csv(data, sep=";")
    return df

def split_data(df):
    y = df['y']
    X = df.drop(columns=['y'])
    return X, y

def YN_numeric_encoding(df, column_name):
    return df[column_name].map({'yes': 1, 'no': 0})
    
def one_hot_encode_column(df, column_name):
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    
    column_data = df[[column_name]]
    
    encoded_data = encoder.fit_transform(column_data)
    
    encoded_columns = encoder.get_feature_names_out([column_name])
    
    encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)
    df_encoded = pd.concat([df, encoded_df], axis=1)
    
    df_encoded = df_encoded.drop(columns=[column_name])
    
    return df_encoded

def standardize_column(df, column_name):
    mean = df[column_name].mean()
    std = df[column_name].std()
    df[column_name] = (df[column_name] - mean) / std
    return df

def months_cyclical_encoding(df):
    column_name ='month'
    month_numeric_dictionary = {
    'dec': 1, 'jan': 12, 'feb': 11, 'mar': 10,
    'apr': 9, 'may': 8, 'jun': 7, 'jul': 6,
    'aug': 5, 'sep': 4, 'oct': 3, 'nov': 2}
    
    df[column_name] = df[column_name].map(month_numeric_dictionary)
    
    df[column_name + '_sin'] = np.sin(2 * np.pi * df[column_name] / 12)
    df[column_name + '_cos'] = np.cos(2 * np.pi * df[column_name] / 12)
    df = df.drop(columns=['month'])  # Drop original month column
    return df

def day_cyclical_encoding(df):
    column_name = 'day'
    # Assuming days are in the range 1-31
    df[column_name + '_sin'] = np.sin(2 * np.pi * df[column_name] / 31)
    df[column_name + '_cos'] = np.cos(2 * np.pi * df[column_name] / 31)
    df = df.drop(columns=[column_name])  # Drop original day column if no longer needed
    return df

# #Maybe use Target Encoding instead of OHE to reduce computational time
# def target_encode_column(df, column_name, target_column):
#     # Calculate mean target for each category
#     means = df.groupby(column_name)[target_column].mean()
#     # Map the mean values to the column
#     df[column_name + "_target_encoded"] = df[column_name].map(means)
#     # Optionally drop the original column
#     df = df.drop(columns=[column_name])
#     return df

# Get Data
url = "https://raw.githubusercontent.com/tuclaure/CSU-CS-345/refs/heads/main/Data/bank/bank-full.csv"
df = fetch_data(url)

#Apply one Hot Encoding
categorical_columns = ['job', 'marital', 'education', 'contact', 'poutcome']
for column in categorical_columns:
    df = one_hot_encode_column(df, column)
    
#Turn Yes/No Columns into binary numbers
yn_columns = ['default','housing','loan','y']
for column in yn_columns:
    df[column] = YN_numeric_encoding(df, column)
  
#Convert months to cyclical representation 
df = months_cyclical_encoding(df)

#Convert days to cyclical representation
df = day_cyclical_encoding(df)

#Standardize necessary columns
std_columns = ['balance', 'duration', 'pdays']
for column in std_columns:
    df = standardize_column(df, column)
    
#Split into features
X, y = split_data(df)



##CURRENT DATA MANIPULATIONS
#One hot code; job, marital, education, contact, poutcome
#Turning Yes/No columns into Binary; default, housing, loan, y
#Cyclical framing of data for; months, days
#standardization of; age, balance, duration, pdays

#Left todo
#PDAYS NEEDS MANIPULATION, -1 is used to indicate never contacted, but our current manipulation doesnt represent this well, look into options
#Potentially use Target Encoding instead of One Hot Encoding


# df.iloc[0]
print(X)
print(y)


       age  default   balance  housing  loan  duration  campaign     pdays  \
0       58        0  0.256416        1     0  0.011016         1 -0.411449   
1       44        0 -0.437890        1     0 -0.416122         1 -0.411449   
2       33        0 -0.446758        1     1 -0.707353         1 -0.411449   
3       47        0  0.047205        1     0 -0.645224         1 -0.411449   
4       33        0 -0.447086        0     0 -0.233618         1 -0.411449   
...    ...      ...       ...      ...   ...       ...       ...       ...   
45206   51        0 -0.176458        0     0  2.791298         3 -0.411449   
45207   71        0  0.120445        0     0  0.768216         2 -0.411449   
45208   72        0  1.429577        0     0  3.373760         5  1.436173   
45209   57        0 -0.228021        0     0  0.970136         4 -0.411449   
45210   37        0  0.528359        0     0  0.399324         2  1.476121   

       previous  job_blue-collar  ...  education_unknown  conta

In [83]:
#Validation of data test

def validate_data(df):
    """
    Validates a DataFrame dynamically for common data issues such as NaNs, 
    inconsistent data types, and extreme outliers based on IQR.

    Parameters:
        df (pd.DataFrame): The DataFrame to validate.
        
    Returns:
        dict: A dictionary with validation results.
    """
    results = {
        'missing_values': None,
        'data_type_mismatches': None,
        'outliers': None
    }
    
    # 1. Check for NaN values in each column
    missing_values = df.isna().sum()
    results['missing_values'] = missing_values[missing_values > 0].to_dict()

    # 2. Check for inconsistent data types
    # For each column, ensure all entries are of the most common type
    data_type_mismatches = {}
    for col in df.columns:
        col_types = df[col].dropna().apply(type).value_counts()
        if len(col_types) > 1:
            data_type_mismatches[col] = col_types.to_dict()
    results['data_type_mismatches'] = data_type_mismatches

    # 3. Check for outliers using IQR (only for numeric columns)
    outliers = {}
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_indices = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index.tolist()
        
        if outlier_indices:
            outliers[col] = outlier_indices
    results['outliers'] = outliers
    
    return results

# Validate the data
validation_results = validate_data(df)

# Print results
print("Validation Results:")
for key, result in validation_results.items():
    print(f"{key}: {result}")
    

# Assuming df is your DataFrame and it has a column named "age"
def calculate_average_age(df):
    # Check if 'age' column exists in the DataFrame
    if 'age' in df.columns:
        # Calculate and return the mean of the 'age' column, excluding any NaN values
        average_age = df['age'].mean()
        print(f"Average Age: {average_age}")
    else:
        print("The 'age' column does not exist in the DataFrame.")
        
calculate_average_age(df)

Validation Results:
missing_values: {}
data_type_mismatches: {}
outliers: {'age': [29158, 29261, 29263, 29322, 29865, 30225, 30264, 30908, 31051, 31052, 31055, 31057, 31069, 31077, 31089, 31099, 31101, 31108, 31116, 31125, 31130, 31145, 31149, 31151, 31183, 31189, 31233, 31255, 31261, 31285, 31341, 31365, 31383, 31393, 31395, 31414, 31447, 31527, 31569, 31647, 31723, 31816, 32009, 32032, 32064, 32068, 32074, 32089, 32105, 32160, 32204, 32321, 32451, 32689, 32699, 32802, 33017, 33360, 33414, 33455, 33699, 33706, 33710, 33719, 33729, 33751, 33752, 33766, 33775, 33777, 33780, 33803, 33810, 33811, 33815, 33827, 33846, 33860, 34128, 40363, 40367, 40368, 40374, 40381, 40383, 40389, 40401, 40402, 40412, 40471, 40481, 40515, 40524, 40526, 40538, 40561, 40569, 40571, 40584, 40586, 40595, 40621, 40645, 40655, 40657, 40674, 40676, 40688, 40699, 40702, 40710, 40712, 40726, 40732, 40759, 40766, 40786, 40789, 40792, 40816, 40837, 40838, 40843, 40851, 40864, 40872, 40896, 40910, 40913, 40932, 40947, 